## 2. Trimming fastqs

fastp is a brand new fastq preprocessor that is "2~5 times faster than other FASTQ preprocessing tools like Trimmomatic or Cutadapt". Automatically finds and trims adapters and quality filters. See [publication](https://academic.oup.com/bioinformatics/article/34/17/i884/5093234) and [documentation](https://github.com/OpenGene/fastp) for more info. 

In [1]:
##conda install -c bioconda fastp
import pandas as pd
import os
import subprocess

In [3]:
df = pd.read_csv("/moto/eaton/projects/macaques/metadata.csv")
df[["Species", "Group", "SRR", "BioSample", "Sample", "Study", "PRJ"]]

,Species,Group,SRR,BioSample,Sample,Study,PRJ
0,Macaca mulatta northern low altitude,mulatta,SRR4454026,SAMN05883679,SRS1762015,SRP092140,PRJNA345528
1,Macaca mulatta southern high altitude,mulatta,SRR4454020,SAMN05883709,SRS1762009,SRP092140,PRJNA345529
2,Macaca mulatta southern low altitude,mulatta,SRR4453966,SAMN05883736,SRS1761955,SRP092140,PRJNA345530
3,Macaca mulatta Indian,mulatta,SRR5628058,SAMN07168901,SRS2238957,SRP049547,PRJNA251548
4,Macaca fascicularis northern,fascicularis,fasno,SAMN00116341,SRS117874,SRP045755,PRJNA51411
5,Macaca fascicularis southern,fascicularis,SRR445713,SAMN00811240,SRS300124,SRP011089,PRJNA20409
6,Macaca fuscata,mulatta,DRR002233,SAMD00011919,DRS001583,DRP000620,PRJDB2459
7,Macaca fuscata,mulatta,fuscata,SAMD00013516,DRS002017,DRP000657,PRJDB2648
8,Macaca thibethana,sinica,SRR1024051,SAMN02390221,SRS498543,SRP032525,PRJNA226187
9,Macaca assamensis,sinica,SRR2981114,SAMN04316321,SRS1196892,SRP067118,PRJNA305009


1) Filtering 6 of the species default arguments (other than thread count - set to 12):

In [5]:
!mkdir /moto/eaton/projects/macaques/filteredfastq

In [3]:
!mkdir /moto/eaton/projects/macaques/filteredfastq/stats

In [5]:
batch1 = ['fasno', 'fuscata', 'nemestrina', 'silenus', 'sylvanus']

In [ ]:
batch2 = ['SRR4454026', 'SRR4454020', 'SRR4453966', 'SRR5628058', 'SRR445713', 'DRR002233',
          'SRR1024051', 'SRR2981114', 'SRR2981139', 'SRR2981140', 'SRR5947292', 'SRR5947293', 
          'SRR5947294', 'SRR7588781', 'SRR8285768']

In [ ]:
for i in batch1:
    cmd='fastp -i /moto/eaton/projects/macaques/fastqdump/'+i+'.sra_1.fastq.gz \
            -I /moto/eaton/projects/macaques/fastqdump/'+i+'.sra_2.fastq.gz \
            -o /moto/eaton/projects/macaques/filteredfastq/'+i+'.filtered_1.fastq.gz \
            -O /moto/eaton/projects/macaques/filteredfastq/'+i+'.filtered_2.fastq.gz \
            -w 12 \
            --json /moto/eaton/projects/macaques/filteredfastq/stats/'+i+'.json \
            --html /moto/eaton/projects/macaques/filteredfastq/stats/'+i+'.html'
    os.system(cmd)

In [ ]:
for i in batch2:
    cmd='fastp -i /moto/eaton/projects/macaques/fastqdump/'+i+'.sra_1.fastq.gz \
            -I /moto/eaton/projects/macaques/fastqdump/'+i+'.sra_2.fastq.gz \
            -o /moto/eaton/projects/macaques/filteredfastq/'+i+'.filtered_1.fastq.gz \
            -O /moto/eaton/projects/macaques/filteredfastq/'+i+'.filtered_2.fastq.gz \
            -w 12 \
            --json /moto/eaton/projects/macaques/filteredfastq/stats/'+i+'.json \
            --html /moto/eaton/projects/macaques/filteredfastq/stats/'+i+'.html'
    os.system(cmd)

In [6]:
##Summary of filtering/trimming can be viewed using the below
from IPython.display import HTML
HTML(filename='/moto/eaton/projects/macaques/filteredfastq/stats/fasno.html')

fastp version:,0.20.0 (https://github.com/OpenGene/fastp)
sequencing:,paired end (75 cycles + 75 cycles)
mean length before filtering:,"54bp, 54bp"
mean length after filtering:,"53bp, 53bp"
duplication rate:,19.429169%
Insert size peak:,49
total reads:,3.488583 G
total bases:,191.154732 G
Q20 bases:,122.590192 G (64.131393%)
Q30 bases:,41.044995 G (21.472131%)
GC content:,42.511113%
